In [22]:
import pandas as pd
import numpy as np
import os
import calc
import draw
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import Image
from itertools import product

DATA_PATH = "data"
# check if rendering within sections folder
if os.getcwd().split("/")[-1] == "notebooks":
    DATA_PATH = "../" + DATA_PATH

def clean_state(s):
    if "," in s:
        s = s.replace(",", "")
    if "/" in s:
        s = s.split("/")[0]
    if s == "Washington D.C.":
        s = "District of Columbia"
    return s.strip()


In [23]:
state_dict = calc.get_state_dict()
state_inv_dict = {v: k for k, v in state_dict.items()}
acc_dict = calc.get_state_dict(acc=True)
month_dict = {
    1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
}

In [24]:
rand = pd.read_parquet(f"{DATA_PATH}/nets/rand.parquet")
lei = pd.read_parquet(f"{DATA_PATH}/nets/lei.parquet")
noadj = pd.read_parquet(f"{DATA_PATH}/nets/noadj.parquet")
gemini = pd.read_parquet(f"{DATA_PATH}/nets/gemini-2.5-flash-lite-preview-06-17.parquet")
gpt = pd.read_parquet(f"{DATA_PATH}/nets/gpt-4.1-nano-2025-04-14.parquet")

data_dfs = [rand, lei, noadj, gemini, gpt]
data_names = [
    "Random",
    "Mobility data",
    "Mobility data (alternative)",
    "Gemini 2.5 Flash Lite",
    "GPT-4.1 Nano",
]

In [25]:
corr_df = pd.DataFrame(
    np.eye(len(data_dfs), dtype=object),
)
for i in range(len(data_dfs)):
    for j in range(i + 1, len(data_dfs)):
        corr = calc.gcor(data_dfs[i], data_dfs[j])
        corr_df.iloc[j, i] = f"{corr:.3f}"
        corr_df.iloc[i, j] = ""

    corr_df.iloc[i, i] = "–"

In [26]:
corr_tbl = corr_df.copy()
corr_labels = [f"({i + 1})" for i in range(len(data_dfs))]
corr_tbl.index = corr_labels
corr_tbl.index.name = None
corr_tbl.columns = corr_labels
corr_tbl.insert(0, "Networks", data_names)
corr_tbl

,Networks,(1),(2),(3),(4),(5)
(1),Random,–,,,,
(2),Mobility data,0.082,–,,,
(3),Mobility data (alternative),0.114,0.951,–,,
(4),Gemini 2.5 Flash Lite,0.096,0.933,0.897,–,
(5),GPT-4.1 Nano,0.070,0.975,0.917,0.942,–


In [27]:
with open(f"../tables/_tbl-corr.qmd", "w") as f:
    f.write(
        corr_tbl.to_markdown(
            index=True,
            colalign=["left"]*2 + ["right"] * 5,
            tablefmt="pipe"
        )
    )


In [28]:
desc_df = calc.long_data(data_dfs, calc.dst_by_month, data_names)

In [29]:
unique_data = desc_df["data"].unique()[1:]
n = len(unique_data)

fig = make_subplots(
    rows=2,
    cols=2 * n,
    column_widths=[0.7, 0.3] * n,
    row_heights=[0.1, 0.9],
    specs=[[{"type": "bar"}, None] * n, [{"type": "heatmap"}, {"type": "bar"}] * n],
    horizontal_spacing=0.01,
    vertical_spacing=0.01,
    subplot_titles=[str(name) for name in unique_data],
)
for annotation in fig.layout.annotations:
    annotation.yanchor = 'bottom'
    annotation.y += 0.01  # Adjust the vertical position of the annotation
r_min, r_max = desc_df.groupby(["data", "dst"])["prop"].sum().agg(["min", "max"])

for idx, name in enumerate(unique_data):
    dst_month_pct = desc_df[desc_df["data"] == name]
    month_pct = dst_month_pct.groupby("month")["prop"].sum()
    dst_pct = dst_month_pct.groupby("dst")["prop"].sum()

    col = 2 * idx + 1  # column index starts from 1

    # Heatmap
    fig.add_trace(
        go.Heatmap(
            y=dst_month_pct["dst"].map(state_dict),
            x=dst_month_pct["month"].map(month_dict),
            z=dst_month_pct["prop"],
            coloraxis="coloraxis",
            hovertemplate="%{y}<br>" + "%{x}<br>" + "%{z:.2%}<extra></extra>",
        ),
        row=2,
        col=col,
    )

    # Top bar chart
    fig.add_trace(
        go.Bar(
            x=month_pct.index.map(month_dict),
            y=month_pct.values,
            marker_color="gray",
            showlegend=False,
            yaxis="y2",
            hovertemplate="%{x}<br>" + "%{y:.2%}<extra></extra>",
        ),
        row=1,
        col=col,
    )

    # Right-side horizontal bar chart
    fig.add_trace(
        go.Bar(
            y=dst_pct.index.map(state_dict),
            x=dst_pct.values,
            orientation="h",
            marker_color="gray",
            showlegend=False,
            hovertemplate="%{y}<br>" + "%{x:.2%}<extra></extra>",
        ),
        row=2,
        col=col + 1,
    )

    # Axes formatting
    fig.update_xaxes(tickvals=list(range(0, 13)), tickangle=90, row=2, col=col)
    fig.update_xaxes(showticklabels=False, row=1, col=col)
    fig.update_yaxes(showticklabels=False, row=2, col=col)
    fig.update_yaxes(showticklabels=False, row=2, col=col + 1)
    fig.update_xaxes(range=[r_min, r_max], row=2, col=col + 1, tickformat="0%")

fig.update_yaxes(matches="y2", row=1, tickformat="0%")
fig.update_yaxes(type="category", tickvals=list(range(0, 51)),showticklabels=True, row=2, col=1)
fig.update_yaxes(autorange="reversed", row=2)
fig.update_xaxes(fixedrange=True, mirror=False)
fig.update_yaxes(fixedrange=True, mirror=False)
fig.update_coloraxes(
    colorbar=dict(
        orientation="h",
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        thickness=10,
        len=0.5,
        tickformat="0%",
        title=dict(
                text="Percentage of visits (mean over 1,000 iterations)",
                font=dict(size=16),
                side="top"
        )
    )
)
fig.update_layout(
    height=870,
    width=1100,
    coloraxis={"colorscale": "agsunset_r"},
    margin=dict(t=50, r=50, b=40, l=40),
)
fig.write_image("../figures/fig-month.png", scale=4)
fig.show()

In [30]:
raw_gemini = pd.read_parquet(f"{DATA_PATH}/nets-raw/gemini-2.5-flash-lite-preview-06-17.parquet")
raw_gpt = pd.read_parquet(f"{DATA_PATH}/nets-raw/gpt-4.1-nano-2025-04-14.parquet")
sample = pd.read_parquet(f"{DATA_PATH}/sample.parquet")

In [31]:
new_inc = {
    "less than $25,000": "< $25k",
    "$25,000 to $49,999": "$25k-$49k",
    "$50,000 to $74,999": "$50k-$74k",
    "$75,000 to $99,999": "$75k-$99k",
    "$100,000 to $149,999": "$100k-$149k",
    "$150,000 or more": "≥ $150k",
}
new_age = {
    "18 to 24 years old": "18-24",
    "25 to 34 years old": "25-34",
    "35 to 44 years old": "35-44",
    "45 to 54 years old": "45-54",
    "55 to 64 years old": "55-64",
    "65 years old or older": "65+",
}
sample["incgrp"] = sample["incgrp"].cat.rename_categories(new_inc)
sample["agegrp"] = sample["agegrp"].cat.rename_categories(new_age)

In [32]:
sample = sample.rename(columns={"state": "org"})
sample["org"] = sample["org"].map(acc_dict)

In [33]:
raw_gemini = sample.merge(raw_gemini, on=["iter", "sampleid"], how="right")
raw_gpt = sample.merge(raw_gpt, on=["iter", "sampleid"], how="right")
raw_gemini["state"] = raw_gemini["state"].apply(clean_state)
raw_gpt["state"] = raw_gpt["state"].apply(clean_state)
raw_gemini["state_acc"] = raw_gemini["state"].map(state_inv_dict).map(acc_dict)
raw_gpt["state_acc"] = raw_gpt["state"].map(state_inv_dict).map(acc_dict)

In [34]:
raw_gpt["location"] = raw_gpt["location"].map(lambda x: x.split(", ")[0] if ", " in x else x)

In [35]:
# n missings
raw_gemini["state_acc"].isna().sum(), raw_gpt["state_acc"].isna().sum()

(np.int64(531), np.int64(823))

In [36]:
# missing examples
raw_gemini[raw_gemini["state_acc"].isna()][["location", "state"]].value_counts().head(10)

location             state           
Montreal             Quebec              302
Banff National Park  Alberta              61
San Juan             Puerto Rico          32
Vancouver            British Columbia     29
Toronto              Ontario              17
Paris                France               14
Banff National Park  Alberta Canada       14
Banff                Alberta              10
Montreal             Quebec Canada         8
London               United Kingdom        5
Name: count, dtype: int64

In [37]:
raw_gemini[raw_gemini["state_acc"].isna()]["iter"].value_counts()

iter
906    7
603    6
282    5
894    5
868    5
      ..
354    1
353    1
352    1
349    1
998    1
Name: count, Length: 361, dtype: int64

In [38]:
raw_gpt[raw_gpt["state_acc"].isna()][["location", "state"]].value_counts().head(10)

location                state                   
San Juan                Puerto Rico                 272
Banff National Park     Alberta                     103
Victoria                British Columbia             30
                                                     20
Banff                   Alberta                      14
Vancouver               British Columbia             11
Lake Tahoe              California Nevada            10
Puerto Rico (San Juan)  Puerto Rico                   6
Lake Tahoe              California Nevada border      6
Banff National Park     Alberta (Canada)              6
Name: count, dtype: int64

In [39]:
raw_gpt[raw_gpt["state_acc"].isna()]["iter"].value_counts()

iter
50     22
80      6
174     6
23      6
32      5
       ..
387     1
385     1
381     1
379     1
999     1
Name: count, Length: 525, dtype: int64

In [19]:
raw_gemini = raw_gemini.dropna(subset=["state_acc"])
raw_gpt = raw_gpt.dropna(subset=["state_acc"])

In [20]:
raw_gemini["location"] = raw_gemini.apply(lambda x: x["location"] + ", " + x["state_acc"], axis=1)
raw_gpt["location"] = raw_gpt.apply(lambda x: x["location"] + ", " + x["state_acc"], axis=1)

In [21]:
gpt_top10 = raw_gpt["location"].value_counts().head(10).to_frame().reset_index()
gpt_top10["count"] = gpt_top10["count"].map(lambda x: f"{x:,}")
gemini_top10 = raw_gemini["location"].value_counts().head(10).to_frame().reset_index()
gemini_top10["count"] = gemini_top10["count"].map(lambda x: f"{x:,}")

In [22]:
top10 = pd.concat([gemini_top10, gpt_top10], axis=1)
top10.index = top10.index + 1
top10.index.name = "Rank"
top10.columns = ["Gemini 2.5 Flash Lite", "Freq.", "GPT-4.1 Nano", "Freq."]
top10

,Gemini 2.5 Flash Lite,Freq.,GPT-4.1 Nano,Freq.
Rank,,,,
1,"Maui, HI","56,296","Sedona, AZ","31,874"
2,"Chicago, IL","38,199","Savannah, GA","27,375"
3,"San Francisco, CA","32,644","Asheville, NC","25,400"
4,"Aspen, CO","32,328","Santa Barbara, CA","25,092"
5,"Austin, TX","32,006","Napa Valley, CA","24,288"
6,"Asheville, NC","31,145","Charleston, SC","22,294"
7,"New York City, NY","29,045","Austin, TX","22,201"
8,"Napa Valley, CA","26,392","San Francisco, CA","21,008"
9,"Boston, MA","24,369","San Antonio, TX","20,696"


In [23]:
with open(f"../tables/_tbl-top10.qmd", "w") as f:
    f.write(
        top10.to_markdown(
            index=True,
            colalign=["right"]*5,
            tablefmt="pipe"
        )
    )

In [24]:
def get_freq(df, col):
    freq = (
        df.groupby([col], observed=True)["location"]
        .value_counts()
        .groupby(col, observed=True)
        .head(1)
    )
    return freq

In [25]:
freqs = []
for df in [raw_gemini, raw_gpt]:
    freq = pd.concat([get_freq(df, col) for col in ["sex", "agegrp", "incgrp"]]).reset_index()
    freq["count"] = freq["count"].map(lambda x: f"{x:,}")
    freqs.append(freq)

freqs = pd.concat(freqs, axis=1)

freqs.columns = ["cat"] + ["Gemini 2.5 Flash Lite", "Freq."] + ["drop"] + ["GPT 4.1 Nano", "Freq."]
freqs["cat"] = freqs["cat"].str.capitalize()
freqs = freqs.rename(columns={"cat": ""})
freqs = freqs.drop(columns=["drop"])
desc_col = (
    ["Sex"]
    + [""] * 1
    + ["Age"]
    + [""] * 5
    + ["Household income"]
    + ["(in 2023 USD)"]
    + [""] * 4
)
freqs.index = desc_col
freqs

,,Gemini 2.5 Flash Lite,Freq.,GPT 4.1 Nano,Freq.
Sex,Female,"Maui, HI","31,465","Sedona, AZ","18,708"
,Male,"Maui, HI","24,831","Sedona, AZ","13,166"
Age,18-24,"Maui, HI","7,156","Asheville, NC","3,414"
,25-34,"Maui, HI","10,144","Austin, TX","5,750"
,35-44,"Maui, HI","8,897","Napa Valley, CA","5,577"
,45-54,"Maui, HI","8,611","Napa Valley, CA","5,783"
,55-64,"Maui, HI","10,853","Sedona, AZ","5,071"
,65+,"Maui, HI","10,635","Sedona, AZ","9,595"
Household income,< $25k,"Asheville, NC","3,724","Asheville, NC","4,688"
(in 2023 USD),$25k-$49k,"Asheville, NC","7,072","Asheville, NC","5,118"


In [26]:
with open(f"../tables/_tbl-freqs.qmd", "w") as f:
    f.write(
        freqs.to_markdown(
            index=False,
            colalign=["left"]*1 + ["right"] * 4,
            tablefmt="pipe"
        )
    )